# HW6: Variational Autoencoders


**STATS271/371: Applied Bayesian Statistics**

_Stanford University. Spring, 2021._

---

**Name:** _Your name here_

**Names of any collaborators:** _Names here_

*Due: 11:59pm Monday, May 17, 2021 via GradeScope*

---

Our goal is to learn the parameters $\Theta$ of a distribution over data, $p(x_n \mid \Theta)$. We assume a latent variable model of the following form:
$$
p(x_n \mid \Theta) = \int p(x_n \mid z_n, \Theta) \, p(z_n) \mathrm{d} z_n
$$
The prior $p(z_n)$ can be very simple (e.g. a standard Gaussian distribution) as long as the likelihood, $p(x_n \mid z_n, \Theta)$, is sufficiently flexible. Then the latent variable model can capture very complex data distributions.

Variational autoencoders (VAEs) are one way of training latent variable models like the one above. We'll build a very simple VAE in this homework assignment and apply it to a standard image dataset, the MNIST dataset of handwritten digits.

Assume the following functional forms for the latent variable model above,
\begin{align}
p(z_n) &= \mathcal{N}(z_n \mid 0, I) \\
p(x_n \mid z_n, \Theta) &= \mathcal{N}(f(z_n, w), \Sigma)
\end{align}
where $f$ is a nonlinear mapping from latent variables $z$ to expected observations $\mathbb{E}[x_n \mid z_n] = f(z_n, w)$. The full set of generative model parameters are $\Theta = (w, \Sigma)$. 

We'll use variational expectation-maximization (vEM) to learn the model parameters. This entails an inner inference step (the variational E-step) to approximate the posterior
\begin{align}
q(z_n; \lambda_n) &\approx p(z_n \mid x_n, \Theta).
\end{align}
Optimizing these variational parameters $\lambda_n$ for each data point can be very time consuming, involving many iterations of gradient ascent for each variational E-step.

The key insight of VAEs is that our time might be better spent optimizing the model parameters instead, and that we can get by with a worse approximation to the posterior if it allows us more updates of $\Theta$. To that end, VAEs simultaneously train an *inference network* to quickly map data points $x_n$ to variational parameters $\lambda_n$. Concretely, VAEs learn a function,
\begin{align}
\lambda_n &= g(x_n; \phi),
\end{align}
The variational parameters $\phi$ are shared by all datapoints, thereby *amortizing* the cost of inference across examples. Under this formulation, we will simply write,
\begin{align}
q(z_n; \lambda_n) &= q(z_n; g(x_n; \phi)) \triangleq q(z_n; x_n, \phi).
\end{align}

To train a variational autoencoder, we perform stochastic gradient ascent on the ELBO $\mathcal{L}(\Theta, \phi)$ with respect to both $\Theta$ and $\phi$ using Monte Carlo estimates of the gradient.

## Problem 1: Math

Consider a dataset $\{x_n\}_{n=1}^N$ with $x_n \in \mathbb{R}^D$ and assume continuous latent variables $z_n \in \mathbb{R}^P$ with a standard normal prior. We'll assume a variational approximation to the posterior on $z_n \in \mathbb{R}^P$ of the form,
\begin{align}
q(z_n; x_n, \phi) &= \mathcal{N}(z_n; \mu_n, \mathrm{diag}(\sigma_n^2)),
\end{align}
where $\mu_n \in \mathbb{R}^P$ and $\sigma_n^2 \in \mathbb{R}_+^P$ are the variational parameters output by the inference network $g(x_n; \phi)$.

### Problem 1a: Write a Monte Carlo estimator for the ELBO 
Use random mini-batches of data to write a Monte Carlo estimate of the ELBO,
\begin{align}
\mathcal{L}(\Theta, \phi) &\approx \ldots
\end{align}

### Problem 1b: Write a Monte Carlo estimate of the gradient wrt $\Theta$
Use random mini-batches of data to write a Monte Carlo estimate of the gradient,
\begin{align}
\nabla_\Theta \mathcal{L}(\Theta, \phi) &\approx \ldots
\end{align}

### Problem 1c: Derive the KL divergence between two Gaussians
Derive the KL divergence between two multivariate normal distributions,
\begin{align}
\mathrm{KL}\big(\mathcal{N}(\mu_1, \Sigma_1) \, \| \, \mathcal{N}(\mu_2, \Sigma_2) \big) &= 
\end{align}

### Problem 1d: Write a Monte Carlo estimate of the gradient wrt $\phi$
Use reparameterization gradients and random mini-batches of data to write a Monte Carlo estimate of the gradient,
\begin{align}
\nabla_\phi \mathcal{L}(\Theta, \phi) &\approx \ldots
\end{align}

## Problem 2: Code
In this problem, you will implement a simple VAE model and train it on the MNIST handwritten digits dataset. The inputs 28x28 pixel images, which are flattened to into vectors of dimension $D=784$.  Let both $p(x_n \mid z_n, \Theta)$ and $q(z_n ; x_n, \phi)$ be parametrized by neural networks with one hidden layer that consists of $512$ ReLU neurons and let the dimensionality of the latent space be $P=2$. The weight matrices between the layers should be initialized randomly by sampling from $\mathcal{N}(0, 0.01)$ and the biases should be initially set to zeros. Since the $x_n$'s are continuous but standardized to lie in $[0,1]$, the output layer of the generative network $f(z_n, w)$ should have sigmoidal nonlinearities.

The variational distribution, $q(z_n; x_n, \phi)$, is a diagonal Gaussian distribution, as in Problem 1. The inference network should output a mean vector and a *non-negative* vector of variances for each latent dimension.

### Problem 2a: Build the VAE
Build the VAE described above. There's no "right" way to organize your code, and different deep learning frameworks encourage different types of implementations. In Python, I like to use classes to encapsulate the parameters of the generative and inference networks (i.e. $\Theta$ and $\phi$). The class would expose automatically differentiable functions like `infer`, `generate`, and `compute_elbo` to map data points to posterior parameters, compute the mean of the image given a latent sample, and evaluate a Monte Carlo estimate of the ELBO for a mini-batch of data, respectively. Then you can use stochastic gradient ascent to maximize the ELBO with respect to the parameters.

#### Note on implementation:
- You are free to use any programming language for your implementation.
- We recommend you additionally use a library with support that allows you to perform automatic reverse-mode differentiation which will simplify model development. Both TensorFlow or PyTorch, e.g., have implemented distributions that make it easy to implement reparameterization gradients.
- You are *not* allowed to use any libraries that provide some sort of pre-made implementations of the variational autoencoders. That is, one line implementations like `vae = VAE(...)` are not in the spirit of this assignment.
- For the optimization, we recommend you use one of the popular algorithms such as Adagrad [1] or Adam [2].


#### Note on the Honor Code:
- There are many examples freely available on the internet of similar implementations. If you follow any such sources, you must clearly cite them in your submission. 
- You need to implement the models and the training algorithms using standard libraries (including TensorFlow, PyTorch, Keras, etc.) yourself.

#### References
- [1] John Duchi, Elad Hazan, and Yoram Singer. Adaptive subgradient methods for online learning and stochastic optimization. Journal of Machine Learning Research, 12(Jul):2121–2159, 2011.
- [2] Diederik Kingma and Jimmy Ba. Adam: A method for stochastic optimization. arXiv preprint arXiv:1412.6980, 2014.

### Problem 2b: Train the VAE with stochastic gradient ascent on the ELBO

Train and evaluate your models on the MNIST handwritten digits dataset. The dataset can be downloaded directly from [here](http://yann.lecun.com/exdb/mnist/). Alternatively, many deep learning libraries have utilities to download the dataset into their desired format. (E.g. [PyTorch](https://pytorch.org/vision/stable/datasets.html#mnist), [tf.keras](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/mnist/load_data), and [TensorFlow for R](https://tensorflow.rstudio.com/guide/keras/))


## Problem 3: Analysis

### Problem 3a: Sample from the VAE
Visualize a random sample of $100$ MNIST digits on $10 \times 10$ tile grid (i.e., $10$ rows, $10$ digits per row).
Using your trained models, sample and visualize $100$ digits from each of them in the same manner. To do this, sample $100$ random $z$, then apply the generator network, $p(x_n \mid z_n)$, to produce digit samples. Comment on the results.

### Problem 3b: Visualize the manifold of digits
Since we have specifically chosen the latent space to be 2-dimensional, now you can easily visualize the learned latent manifold of digits:
- Using your pre-trained recognition networks, transform images from the test set to the latent space. Visualize the points in the latent space as a scatter plot, where colors of the points should correspond to the labels of the digits.
- From the previous point, determine the min and max values of $z_1$ and $z_2$. Create a $20 \times 20$ grid that corresponds to $(z_1, z_2)$ values between the min and max. For each $z = (z_1, z_2)$, generate and visualize digits using each of your trained models, and plot each set on a $20 \times 20$ tile grid.

## Problem 4: Discussion

### Problem 4a: Laplace prior on the latents
Suppose we instead used a [Laplace prior](https://en.wikipedia.org/wiki/Laplace_distribution) $z_n \sim \mathrm{Lap}(0, \tau)$ with density
\begin{align}
p(z_n) &= \frac{1}{2\tau}\exp \left\{-\frac{|z_n|}{\tau} \right\}.
\end{align}
Propose a simple reparametrization of the Laplace distribution $z_n = r(\epsilon_n, \tau)$ with $\epsilon_n \sim p(\epsilon)$ for some function $r$ and distribution $p$, suitable for training a VAE.

### Problem 4b: Discrete latent variables
The present model uses continuous latent variables. Where did we use this assumption and what would have to change if we used discrete $z_n$'s instead?

# Submission Instructions


**Formatting:** check that your code does not exceed 80 characters in line width. If you're working in Colab, you can set _Tools &rarr; Settings &rarr; Editor &rarr; Vertical ruler column_ to 80 to see when you've exceeded the limit. 

Download your notebook in .ipynb format and use the following commands to convert it to PDF:
```
jupyter nbconvert --to pdf hw6_yourname.ipynb
```

**Dependencies:**

- `nbconvert`: If you're using Anaconda for package management, 
```
conda install -c anaconda nbconvert
```

**Upload** your .ipynb and .pdf files to Gradescope. 
